In [1]:
# import packages
import numpy as np
# import scipy.integrate as integrate
# from sympy import *
import time

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation

from matplotlib import cm
from matplotlib.ticker import LinearLocator
## set default plot size
plt.rcParams["figure.figsize"] = (10, 7)
## set global font size
plt.rcParams.update({'font.size':14})

# import astropy.units as u
from astropy import constants as const

# Computing Particle Trajectories

This notebook focuses on computing and animating the trajectories of individual particles within our two-hammer potential rather than the full solution $\delta f$. I will do this in two ways:
1. By using the analytical solution to Hamilton's equations, assuming two delta-function perturbations.
2. By numerically solving Hamilton's equations, which won't necessarily assume delta-function perturbations.

# Isothermal Slab Dynamics

We'll simulate the motions of individual stars within a two-hammer, isothermal slab potential. The Hamiltonian can be written as:

$$ \mathcal{H}(J, \theta) = \mathcal{H}_\text{slab}(J) + \phi_\text{hammers}(\theta) = \frac{3a}{2b} (1+bJ)^{2/3} + A_1 \cos(n_1 \theta) \delta(t-t_1) + A_2 \cos(n_2 \theta) \delta(t-t_2). $$

In [17]:
start = time.time()

temp = 0
for i in range((int) (1e8)):
    temp = i
    temp = i + 1
    
print('{:2.2f}'.format(time.time()-start))

14.85
